# Baseline

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

## Загрузка данных

In [8]:
train_df = pd.read_parquet("data/train_data.pqt")
test_df = pd.read_parquet("data/test_data.pqt")

In [20]:
test_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,None,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,None
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290115,299998,month_5,NaN,NaN,NaN,NaN,None,None,None,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{}
290116,299998,month_6,-0.155350,-0.203711,-0.125995,-0.155980,channel_code_9,city_25,city_type_0,index_city_code_30,...,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,None
290117,299999,month_4,-0.145899,-0.173298,-0.125995,-0.145402,channel_code_9,city_6,city_type_0,index_city_code_34,...,-0.028584,0.944497,0.384773,-0.097640,0.550901,0.574354,-0.101369,0.252657,0.440474,{α}
290118,299999,month_5,-0.136402,-0.163874,-0.121475,-0.135876,channel_code_9,city_6,city_type_0,index_city_code_34,...,-0.028584,0.944497,0.384773,-0.047885,0.555057,0.696576,-0.034860,0.253523,0.473441,{α}


In [9]:
train_df.head(3)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}


#### target class - "end_cluster" (df[-1])

Обозначение категориальных признаков

In [11]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",
]

cat_cols_num = []

Удаляем все строки с nan'ами

In [19]:
for x,y in zip(test_df.columns, test_df.isna().sum()):
    print(x, "---", y)

id --- 0
date --- 0
balance_amt_avg --- 29483
balance_amt_max --- 29483
balance_amt_min --- 29483
balance_amt_day_avg --- 29483
channel_code --- 20028
city --- 60612
city_type --- 60992
index_city_code --- 156461
ogrn_days_end_month --- 14278
ogrn_days_end_quarter --- 14278
ogrn_month --- 14278
ogrn_year --- 14278
ft_registration_date --- 14838
max_founderpres --- 161192
min_founderpres --- 161192
ogrn_exist_months --- 14278
okved --- 14479
segment --- 14020
sum_of_paym_2m --- 73461
sum_of_paym_6m --- 73461
sum_of_paym_1y --- 73461
sum_a_oper_1m --- 0
cnt_a_oper_1m --- 95210
sum_b_oper_1m --- 0
cnt_b_oper_1m --- 95210
sum_c_oper_1m --- 0
cnt_c_oper_1m --- 95210
sum_deb_d_oper_1m --- 0
cnt_deb_d_oper_1m --- 95210
sum_cred_d_oper_1m --- 0
cnt_cred_d_oper_1m --- 95210
sum_deb_e_oper_1m --- 0
cnt_deb_e_oper_1m --- 95210
cnt_days_deb_e_oper_1m --- 95210
sum_cred_e_oper_1m --- 0
cnt_cred_e_oper_1m --- 95210
cnt_days_cred_e_oper_1m --- 95210
sum_deb_f_oper_1m --- 0
cnt_deb_f_oper_1m --- 95210

Создаем выборки для валидации и обучения

In [ ]:
to_drop_cols = [
    "id", "date", "end_cluster",
    'cnt_a_oper_1m', "cnt_b_oper_1m", 'cnt_c_oper_1m', 'cnt_deb_d_oper_1m','cnt_cred_d_oper_1m','cnt_deb_e_oper_1m','cnt_days_deb_e_oper_1m','cnt_cred_e_oper_1m','cnt_days_cred_e_oper_1m',
    'cnt_deb_f_oper_1m','cnt_days_deb_f_oper_1m','cnt_cred_f_oper_1m','cnt_days_cred_f_oper_1m','cnt_deb_g_oper_1m',
    'cnt_days_deb_g_oper_1m','cnt_cred_g_oper_1m','cnt_days_cred_g_oper_1m','cnt_deb_h_oper_1m','cnt_days_deb_h_oper_1m','cnt_cred_h_oper_1m','cnt_days_cred_h_oper_1m',
]

In [12]:
X = train_df.drop(to_drop_cols, axis=1)
y = train_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [13]:
for i, col in enumerate(cat_cols):
    for j, col1 in enumerate(x_train.columns):
        if col == col1:
            cat_cols_num.append(j)
cat_cols_num = sorted(cat_cols_num)
cat_cols_num

[4, 5, 6, 7, 10, 11, 16, 17, 89]

## Обучение модели

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [14]:
model = CatBoostClassifier(iterations=2,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(x_train, y_train,
        cat_features=cat_cols_num,
        eval_set=(x_val, y_val),
        verbose=False
)
# make the prediction using the resulting model
# preds_class = model.predict(x_val)
preds_proba = model.predict_proba(x_val)
# print("class = ", preds_class)
print("proba = ", preds_proba)

TypeError: must be real number, not NoneType

Зададим функцию для взвешенной метрики roc auc

In [10]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [11]:
cluster_weights = pd.read_excel("/content/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

Проверка работы модели

In [12]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape

(120000, 17)

In [13]:
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.7435520713146397

## Прогноз на тестовой выборке

In [14]:
test_df.pivot(index="id", columns="date", values="start_cluster").head(3)

date,month_4,month_5,month_6
id,,,
200000,{α},{α},NaN
200001,{α},{α},NaN
200002,{α},{α},NaN


Для того, чтобы сделать прогноз на тестовой выборке, нужно заполнить стартовый кластер. </br>
В качестве базового подхода заполним все стартовые кластеры, самым популярным кластером.

In [15]:
test_df["start_cluster"] = train_df["start_cluster"].mode()[0]
test_df["start_cluster"] = test_df["start_cluster"].astype("category")

In [16]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [17]:
sample_submission_df.shape

(100000, 18)

In [18]:
sample_submission_df.head()

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,200001,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,200002,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,200003,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,200004,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


Для тестовой выборки будем использовать только последний месяц

In [19]:
last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

In [20]:
test_pred_proba = model.predict_proba(last_m_test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [21]:
test_pred_proba_df.shape

(100000, 17)

In [22]:
test_pred_proba_df.head(2)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.018570,0.438989,0.001094,0.003762,0.000294,0.000116,1.297136e-07,0.0,0.000829,0.013894,0.001293,5.290471e-11,0.001651,0.0,0.000015,0.519494,0.0
1,0.195437,0.017019,0.043320,0.000162,0.005102,0.000003,6.688079e-05,0.0,0.017252,0.005322,0.015416,7.473112e-08,0.000532,0.0,0.000225,0.700143,0.0


In [23]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("baseline_submission.csv", index=False)